In [1]:
!pip install cuml-cu12==24.10.* --extra-index-url=https://pypi.nvidia.com
!pip install cudf-cu12==24.10.* --extra-index-url=https://pypi.nvidia.com

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.6/567.6 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 836.6/836.6 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 96.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.1/453.1 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.9/196.9 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.1/515.1 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [1]:
import pandas as pd
import numpy as np
import warnings
import time
from sklearn.model_selection import StratifiedKFold, ParameterGrid
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# CPU 모델
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import NuSVC as SklearnNuSVC

# =========================================================
# 1. 라이브러리 로드 및 GPU 체크
# =========================================================
try:
    import cuml
    from cuml.svm import SVC as cuSVC
    from cuml.svm import LinearSVC as cuLinearSVC
    from cuml.neighbors import KNeighborsClassifier as cuKNN
    print(f"✅ [System] cuML(GPU) 로드 성공 (Version: {cuml.__version__})")
    GPU_AVAILABLE = True
except:
    print("⚠️ [System] cuML을 찾을 수 없습니다. CPU로 동작합니다.")
    GPU_AVAILABLE = False

warnings.filterwarnings('ignore')

# ==========================================
# 2. 데이터 로드 및 전처리
# ==========================================
file_path = "Dry_Eye_Dataset_preprocessed.csv"
target_col = 'Dry Eye Disease'

print("\n>> [Data] 데이터 로딩 중...")

try:
    df = pd.read_csv(file_path)
    if df.isnull().sum().sum() > 0: df.fillna(0, inplace=True)

    X_raw = df.drop(columns=[target_col])
    y_raw = df[target_col]

    le = LabelEncoder()
    y_encoded = le.fit_transform(y_raw)

    # GPU 최적화 (float32 + 연속 메모리)
    X = np.ascontiguousarray(X_raw.values, dtype=np.float32)
    y = np.ascontiguousarray(y_encoded, dtype=np.float32)

    # CPU용 y (정수형)
    y_int = y.astype(np.int32)

    print(f"   - 데이터 준비 완료: {X.shape}")

except FileNotFoundError:
    print(f"[Error] 파일을 찾을 수 없습니다. 테스트용 랜덤 데이터를 생성합니다.")
    X = np.random.randn(1000, 20).astype(np.float32)
    y = np.random.randint(0, 2, 1000).astype(np.float32)
    y_int = y.astype(np.int32)

# ==========================================
# 3. 통합 튜닝 함수 (실시간 출력 기능 포함)
# ==========================================
def run_tuning(model_name, model_class, param_grid, device, X, y, cv):
    print(f"\n" + "="*80)
    print(f" 🚀 Running: {model_name} ({device})")
    print(f"="*80)

    start_time = time.time()
    best_score = -1
    best_params = None

    # 파라미터 조합 생성
    param_list = list(ParameterGrid(param_grid))
    total_params = len(param_list)

    for i, params in enumerate(param_list):
        fold_scores = []
        try:
            # Cross Validation Loop
            for train_idx, val_idx in cv.split(X, y):
                # 데이터 분할
                X_train = np.ascontiguousarray(X[train_idx])
                X_val = np.ascontiguousarray(X[val_idx])
                y_train = np.ascontiguousarray(y[train_idx])
                y_val = np.ascontiguousarray(y[val_idx])

                # 모델 학습
                clf = model_class(**params)
                clf.fit(X_train, y_train)

                # 예측
                preds = clf.predict(X_val)

                # 결과 가져오기 (GPU -> CPU)
                if hasattr(preds, 'to_numpy'): preds = preds.to_numpy()
                elif hasattr(preds, 'values'): preds = preds.values.get()
                elif hasattr(preds, 'get'): preds = preds.get()

                fold_scores.append(accuracy_score(y_val, preds))

            # 평균 점수
            avg_score = np.mean(fold_scores)

            # [실시간 출력] 현재 파라미터 점수 출력
            print(f"   [{i+1}/{total_params}] Params: {params}  👉 Acc: {avg_score:.4f}")

            # Best 갱신
            if avg_score > best_score:
                best_score = avg_score
                best_params = params

        except Exception as e:
            print(f"   [{i+1}/{total_params}] Params: {params}  👉 Failed ({str(e)[:30]})")
            continue

    elapsed = time.time() - start_time

    # [모델별 결산 출력]
    print("-" * 80)
    if best_score != -1:
        print(f" ✅ {model_name} Finished.")
        print(f"    - Best Accuracy: {best_score:.4f}")
        print(f"    - Best Params: {best_params}")
    else:
        print(f" ❌ {model_name} Failed (All Error).")
    print("-" * 80)

    return {
        "Model": model_name,
        "Device": device,
        "Best Accuracy": best_score,
        "Best Parameters": best_params,
        "Time (s)": round(elapsed, 2)
    }

# ==========================================
# 4. 실행 설정
# ==========================================
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = []

# --- 모델 설정 ---
# 1. SVC (GPU) - rbf only
svc_params = {'C': [1, 10, 100], 'kernel': ['rbf'], 'gamma': ['scale', 'auto']}

# 2. LinearSVC (GPU)
lsvc_params = {'C': [0.1, 1, 10], 'loss': ['squared_hinge'], 'max_iter': [1000]}

# 3. KNN (GPU)
knn_params = {'n_neighbors': [3, 5, 7, 9]}

# 4. NuSVC (CPU) - rbf only
nusvc_params = {'nu': [0.1, 0.5], 'kernel': ['rbf'], 'gamma': ['scale', 'auto']}

# 5. LDA (CPU)
lda_params = {'solver': ['svd', 'lsqr'], 'shrinkage': [None, 'auto']}

# 6. QDA (CPU)
qda_params = {'reg_param': [0.0, 0.5]}


# --- 실행 루프 ---
if GPU_AVAILABLE:
    results.append(run_tuning("SVC", cuSVC, svc_params, "GPU", X, y, skf))
    results.append(run_tuning("LinearSVC", cuLinearSVC, lsvc_params, "GPU", X, y, skf))
    results.append(run_tuning("KNN", cuKNN, knn_params, "GPU", X, y, skf))

# CPU 모델 실행 (y_int 사용)
results.append(run_tuning("NuSVC", SklearnNuSVC, nusvc_params, "CPU", X, y_int, skf))
results.append(run_tuning("LDA", LinearDiscriminantAnalysis, lda_params, "CPU", X, y_int, skf))
results.append(run_tuning("QDA", QuadraticDiscriminantAnalysis, qda_params, "CPU", X, y_int, skf))

# ==========================================
# 5. 최종 종합 결과 출력
# ==========================================
if results:
    res_df = pd.DataFrame(results).sort_values(by='Best Accuracy', ascending=False).reset_index(drop=True)

    print("\n\n")
    print("#"*80)
    print(" 🏆 FINAL LEADERBOARD 🏆")
    print("#"*80)

    # 전체 순위 출력
    print(res_df[['Model', 'Device', 'Best Accuracy', 'Time (s)']])

    # 1등 강조
    winner = res_df.iloc[0]
    print("\n" + "="*80)
    print(f" 🥇 Absolute Best Model: {winner['Model']} ({winner['Device']})")
    print(f"    Accuracy: {winner['Best Accuracy']:.6f}")
    print(f"    Parameters: {winner['Best Parameters']}")
    print("="*80)

/usr/local/lib/python3.12/dist-packages/cudf/utils/_ptxcompiler.py:64: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/cudadrv/driver.py", line 393, in safe_cuda_api_call
    return self._check_cuda_python_error(fname, libfn(*args))
                                                ^^^^^^^^^^^^
TypeError: cuDriverGetVersion() takes no arguments (1 given)


Not patching Numba
  warnings.warn(msg, UserWarning)
<frozen importlib._bootstrap_external>:1301: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
<frozen importlib._bootstrap_external>:1301: FutureWarning: The cuda.cuda module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.driver module instead.


✅ [System] cuML(GPU) 로드 성공 (Version: 24.10.00)

>> [Data] 데이터 로딩 중...
   - 데이터 준비 완료: (20000, 26)

 🚀 Running: SVC (GPU)
   [1/6] Params: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}  👉 Acc: 0.6870
   [2/6] Params: {'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}  👉 Acc: 0.6621
   [3/6] Params: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}  👉 Acc: 0.6288
   [4/6] Params: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}  👉 Acc: 0.6767
   [5/6] Params: {'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}  👉 Acc: 0.5815
   [6/6] Params: {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}  👉 Acc: 0.5883
--------------------------------------------------------------------------------
 ✅ SVC Finished.
    - Best Accuracy: 0.6870
    - Best Params: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
--------------------------------------------------------------------------------

 🚀 Running: LinearSVC (GPU)
   [1/3] Params: {'C': 0.1, 'loss': 'squared_hinge', 'max_iter': 1000}  👉 Acc: 0.6714
   [2/3] Params: {'C': 1, 'loss': '